In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 29 12:05:29 2020

@author: Rocky
"""
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np 
import pandas as pd

C:\Users\Rocky\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Rocky\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Rocky\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Rocky\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [2]:
def map_size(x_train,x_test):    
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)       
    return x_train,x_test



# input image dimensions
img_rows, img_cols = 28, 28
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()



bool_idx = y_train <5 
x_train_remap = x_train[bool_idx]
y_train_remap = y_train[bool_idx]
bool_idx = y_test <5
x_test_remap = x_test[bool_idx]
y_test_remap = y_test[bool_idx]


batch_size = 128
num_classes = 5
epochs = 30

input_shape = (img_rows, img_cols, 1)
x_train_remap , x_test_remap = map_size(x_train_remap,x_test_remap)

x_train_remap = x_train_remap.astype('float32')
x_test_remap = x_test_remap.astype('float32')
x_train_remap /= 255
x_test_remap /= 255
print('x_train shape:', x_train.shape)
print(x_train_remap.shape[0], 'train samples')
print(x_test_remap.shape[0], 'test samples')
# convert class vectors to binary class matrices
y_train_remap = keras.utils.to_categorical(y_train_remap, num_classes)
y_test_remap = keras.utils.to_categorical(y_test_remap, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train_remap, y_train_remap,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.2)

score = model.evaluate(x_test_remap, y_test_remap, verbose=0)
pre_for_test_set = model.predict(x_test_remap)

print('Test loss:', score[0])
print('Test accuracy:', score[1])








x_train shape: (60000, 28, 28)
30596 train samples
5139 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Train on 24476 samples, validate on 6120 samples
Epoch 1/30
24476/24476 [==============================] - 77s - loss: 0.0408 - acc: 0.8976 - val_loss: 0.0114 - val_acc: 0.9655
Epoch 2/30
24476/24476 [==============================] - 73s - loss: 0.0156 - acc: 0.9587 - val_loss: 0.0087 - val_acc: 0.9739
Epoch 3/30
24476/24476 [==============================] - 73s - loss: 0.0129 - acc: 0.9658 - val_loss: 0.0076 - val_acc: 0.9773
Epoch 4/30
24476/24476 [==============================] - 76s - loss: 0.0115 - acc: 0.9677 - val_loss: 0.0066 - val_acc: 0.9796
Epoch 5/30
24476/24476 [==============================] - 74s 

In [3]:
print('given 0 ~ 4 testing')
bool_idx = y_test <5
x_inference_remap = x_test[bool_idx]
y_inference_remap = y_test[bool_idx]
x_inference_remap = x_inference_remap.reshape(x_inference_remap.shape[0], img_rows, img_cols, 1)
# if not translate data type , what happen?
x_test_remap.astype('float32')
x_test_remap/=255
res = model.predict(x_inference_remap)

print('predict acc ')
decition = np.argmax(res,axis=1)
max_pro = np.max(res,axis=1)
inference_ans1= pd.DataFrame({'label':y_inference_remap,'decition':decition,'max_pro':max_pro})
inference_ans1



given 0 ~ 4 testing
predict acc 


,decition,label,max_pro
0,2,2,1.0
1,1,1,1.0
2,0,0,1.0
3,4,4,1.0
4,1,1,1.0
5,4,4,1.0
6,0,0,1.0
7,0,0,1.0
8,1,1,1.0
9,3,3,1.0


In [4]:
''' given other number test '''
bool_idx = y_test >=5
x_inference_remap = x_test[bool_idx]
y_inference_remap = y_test[bool_idx]
x_inference_remap = x_inference_remap.reshape(x_inference_remap.shape[0], img_rows, img_cols, 1)
# if not translate data type , what happen?
x_test_remap.astype('float32')
x_test_remap/=255
res = model.predict(x_inference_remap)

#print('predict acc ')
decition = np.argmax(res,axis=1)
max_pro = np.max(res,axis=1)
inference_ans2= pd.DataFrame({'label':y_inference_remap,'decition':decition,'max_pro':max_pro})
inference_ans2


,decition,label,max_pro
0,0,7,0.000000e+00
1,4,9,1.000000e+00
2,0,5,0.000000e+00
3,4,9,1.000000e+00
4,0,6,1.000000e+00
5,4,9,1.000000e+00
6,3,5,1.000000e+00
7,4,9,1.000000e+00
8,0,7,0.000000e+00
9,4,9,1.000000e+00
